In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# 提取RESULT文件里的模型名和预测数据
def get_result(file_name,first_):
    model_names,  names = ['auto', 'cross', 'non', 'DL', 'trans', 'pyra', 'times', 'inf', 'light', 'FED'], []
    preds = []
    for num_of_model in range(10):
        files = os.listdir(f'/userhome/cs2/u3604482/capstone/Time-Series-Library-main/RESULTS/{file_name}/results') 
        lastIndex, curIndex, count, name = -1, -1, 0, files[num_of_model]
        preds.append(f'/userhome/cs2/u3604482/capstone/Time-Series-Library-main/RESULTS/{file_name}/results/{name}/pred.npy')
        while count < first_+1:
            lastIndex = curIndex
            curIndex += name.index('_') + 1
            count += 1
            name = files[num_of_model][curIndex+1:]
        if (model_name := files[num_of_model][lastIndex+1:curIndex]) in model_names:
            names.append(model_name)
        else:
            print("WRONG:",model_name)
    return names, preds

# 为每个模型建立预测数据的data frame
def create_pred_df(model, _model_names, low_model_names, trend_model_names, high_preds, low_preds, trend_preds):
    high_model_index = high_model_names.index(model)
    low_model_index = low_model_names.index(model)
    trend_model_index = trend_model_names.index(model)
    high_pred = np.load(high_preds[high_model_index])[:,:,-1]
    low_pred = np.load(low_preds[low_model_index])[:,:,-1]
    trend_pred = np.load(trend_preds[trend_model_index])[:,:,-1]
    preds = {}
    preds[model+'_high'] = high_pred.squeeze()
    preds[model+'_low'] = low_pred.squeeze()
    preds[model+'_trend'] = trend_pred.squeeze()
    preds = pd.DataFrame(preds)
    preds[model+'_pred_close'] = preds[model+'_high'] + preds[model+'_low'] + preds[model+'_trend']
    return preds

high_model_names, high_preds  = get_result('high_ceemdan_pl192_slide0',7)
low_model_names, low_preds = get_result('low_ceemdan_pl192_slide0',7)
trend_model_names, trend_preds = get_result('trend_ceemdan_pl192_slide0',7)

In [ ]:
models = ['auto', 'cross', 'non', 'DL', 'trans', 'pyra', 'light', 'times', 'inf', 'FED']
df_close = pd.read_csv('/userhome/cs2/u3604482/capstone/Time-Series-Library-main/dataset/myData/raw_data/000300.SH.csv')
df = pd.DataFrame([])
pred_len = 192
for model in models:
    df = pd.concat([df,create_pred_df(model, high_model_names, low_model_names, trend_model_names, high_preds, low_preds, trend_preds)], axis=1)
df['next_close'] = df_close.next_close[-pred_len:].reset_index(drop=True)

In [ ]:
df.head()